In [3]:
import requests
from bs4 import BeautifulSoup as bs

In [4]:
kinds = ['book','blog','news', 'shop']
prompt = '''
    1. Book Search
    2. Blog Search
    3. News Search
    4. Shopping Search
    5. Quit
'''

In [5]:
def naver_search_xml(number, query, display, start, sort): #, dtype
#     types = ['xml', 'json']
    client_id = "Cu_DbXmzUvA5GQoVwtnG"
    client_secret = "qUeRsSEWAf"
    url = f'https://openapi.naver.com/v1/search/{kinds[int(number)-1]}.xml' # {types[int(dtype)-1]}
    params = {'query': query, 'display': display, 'start': start, 'sort': sort}
    headers = {'X-Naver-Client-Id': client_id, 'X-Naver-Client-Secret': client_secret}
    resp = requests.get(url, headers=headers, params=params)
    if resp.status_code == 200:
        return [resp.text, resp.status_code]
    else:
        return ['잘못된 요청입니다.', resp.status_code]

In [6]:
def get_book_list(resp):
    soup = bs(resp)
    title_list = list(map(lambda i:i.text.replace('<b>','').replace('</b>',''), soup.select('item>title')))
    price_list = list(map(lambda i:i.text, soup.select('item>price')))
    discount_list = list(map(lambda i:i.text, soup.select('item>discount')))
    book_list = [(t, p, d) for t, p, d in zip(title_list, price_list, discount_list)]
    return book_list

In [7]:
def get_blog_list(resp):
    soup = bs(resp)
    title_list = list(map(lambda i:i.text.replace('<b>','').replace('</b>',''), soup.select('item>title')))
    name_list = list(map(lambda i:i.text, soup.select('item>bloggername')))
    link_list = list(map(lambda i:i.text, soup.select('item>bloggerlink')))
    blog_list = [(t, n, l) for t, n, l in zip(title_list, name_list, link_list)]
    return blog_list

In [8]:
def get_news_list(resp):
    soup = bs(resp)
    title_list = list(map(lambda i:i.text.replace('<b>','').replace('</b>',''), soup.select('item>title')))
    olink_list = list(map(lambda i:i.text, soup.select('item>originallink')))
    link_list = list(map(lambda i:i.next, soup.select('item>link')))
    news_list = [(t, o, l) for t, o, l in zip(title_list, olink_list, link_list)]
    return news_list

In [9]:
def get_shop_list(resp):
    soup = bs(resp)
    title_list = list(map(lambda i:i.text.replace('<b>','').replace('</b>',''), soup.select('item>title')))
    price_list = list(map(lambda i:i.text, soup.select('item>lprice')))
    mall_list = list(map(lambda i:i.text, soup.select('item>mallname')))
    shop_list = [(t, p, m) for t, p, m in zip(title_list, price_list, mall_list)]
    return shop_list

In [14]:
def app():
#     prompt2 = '''
#         데이터 출력 선택
#         1. xml
#         2. json
#     '''
    number = 0
    while True:

        print(prompt)
        number = input()
        if number == '5':
            print('종료')
            break
        query = input('검색어를 넣으세요 >')
#         dtype = input(prompt2)
        display = 10
        start = 1
        sort = 'date'
        resp = naver_search_xml(number, query, display, start, sort)# , dtype
        if resp[1] == 200:
            resp = resp[0]
        else:
            print(resp[0])
            continue
        list = []
        if number == '1':
            list = get_book_list(resp)
        elif number == '2':
            list = get_blog_list(resp)
        elif number == '3':
            list = get_news_list(resp)
        elif number == '4':
            list = get_shop_list(resp)
        print(f'\n{kinds[int(number)-1]} 검색 keyword : {query}\n')
        for i in range(len(list)):
            print(f'*************************{i+1}*************************')
            for item in list[i]:
                print(item)

In [16]:
# app()

In [130]:
resp = naver_search_xml(3, 'bigdata', display, start, sort)# , dtype
soup = bs(resp[0])
title_list = list(map(lambda i:i.text.replace('<b>','').replace('</b>',''), soup.select('item>title')))
olink_list = list(map(lambda i:i.text, soup.select('item>originallink')))
link_list = list(map(lambda i:i.text, soup.select('item')))
news_list = [(t, o, l) for t, o, l in zip(title_list, olink_list, link_list)]
sel = soup.select_one('item>link')